In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
from tqdm import tqdm

from selenium.webdriver.common.by import By #python ver.3.7 이상일 경우 추가 필요

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')

# url : 원하는 어플 url 주소로 바꿔주면 됩니당
url = 'https://play.google.com/store/apps/details?id=com.kakaobank.channel&hl=ko&gl=US'

driver.get(url)

time.sleep(5)

### 리뷰 더보기 창 들어가기

In [ ]:
driver.find_element(By.XPATH,r'//*[@id="yDmH0d"]/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/header/div/div[2]/button').click() #python ver.3.7 이상: driver.find_element(By.XPATH,

In [ ]:
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

In [ ]:
while True:        
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1,1.7))
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:   
        break

    last_height = new_height

driver.execute_script("window.scrollTo(0, 0)")

In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight") 

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(random.uniform(1,1.7))
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
        
    last_height = new_height

In [ ]:
scroll_location = driver.execute_script("return document.body.scrollHeight")

In [ ]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
scroll_height = driver.execute_script("return document.body.scrollHeight")
scroll_height

In [ ]:
#스크롤 내리기 이동 전 위치
scroll_location = driver.execute_script("return document.body.scrollHeight")

while True:
	#현재 스크롤의 가장 아래로 내림
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
       
    #전체 스크롤이 늘어날 때까지 대기
    time.sleep(2)
    
    #늘어난 스크롤 높이
    scroll_height = driver.execute_script("return document.body.scrollHeight")

    #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
    if scroll_location == scroll_height:
    	break
	    
    #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
    else:
    	#스크롤 위치값을 수정
    	scroll_location = driver.execute_script("return document.body.scrollHeight")

In [ ]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [ ]:
#크롤링 관련 코드
soup = bs(driver.page_source, 'lxml')

date_list = []
name_list = []
star_list = []
review_list = []
like_list = []

In [ ]:
#이름 크롤링
names = soup.find_all('div', class_='X5PpBb')

for name in names:
    name_list.append(name.text)
name_list

In [ ]:
#날짜 크롤링
dates = soup.find_all('span', class_='bp9Aid')

for date in dates:
    date_list.append(date.text)
date_list

In [ ]:
#별점 크롤링
stars = soup.find_all('div', class_='iXRFPc')

for star in stars:
    z = re.findall(r'\d{1}개{1}', str(star))
    z = int(z[1][0]) 
    star_list.append(z)
star_list

In [ ]:
# 댓글 크롤링
reviews = soup.find_all('div', class_='h3YV2d')

for review in reviews:
    review_list.append(review.text)
review_list

In [ ]:
name_list = pd.DataFrame(name_list)
date_list = pd.DataFrame(date_list)
star_list = pd.DataFrame(star_list)
review_list = pd.DataFrame(review_list)
df = pd.concat([date_list, star_list, review_list], axis = 1)


df.columns = ['Date', 'Star', 'Review']

# useless = df[df['Name'] == 'KakaoBank Corp.'].index # 어플 관리자 답변 리뷰 삭제
# df = df.drop(useless)
# df = df.reset_index(drop=True)

In [ ]:
df